In [86]:
%load_ext cython

The cython extension is already loaded. To reload it, use:
  %reload_ext cython


In [93]:
import sys
sys.path.append("/home/olek/semestr_7/engineer/py")

# Naive algorithm

In [94]:
%%cython
import numpy as np
import random 
from datetime import datetime

from bernstein_race.IEBLR import IEBLR

cdef float epsilon, delta, sig
cdef int n_max, n_jobs, i_max

epsilon = 0.01
delta = 0.05

def alg1(x0, symulation, delta = 0.01, epsilon = 0.05, n_max = 208, n_jobs = 4, i_max = None):
    cdef int n, i, iterations 
    sig = 1
    x0 = np.array(x0)
    x = x0
    n = len(x0)
    i = 0
    iterations = 0
    if not i_max:
        i_max = 0
    with open("x_history", "w+") as file:
        while 1:
            child = [min(700, x_i + sig * np.random.normal()) for x_i in x] #python limitation
            race1 = IEBLR(symulation, x, x0, 
                            delta = delta, epsilon = epsilon, n_max = n_max, n_jobs = n_jobs)
            race2 = IEBLR(symulation, child, x0, 
                            delta = delta, epsilon = epsilon, n_max = n_max, n_jobs = n_jobs)
            if race2 > race1:
                x = child
                sig = min(10, 2*sig) #
                today = datetime.now()
                ps = np.exp(x)/np.sum(np.exp(x))
                i+=1
                file.write(f"{i}, {x}, {today} \n")
            else:
                sig = 0.84*sig
            iterations+=1
            if iterations > i_max:
                break   

In [89]:
from games.war3par.war import symulation
alg1([1,1,1], symulation, i_max = 50)

In [90]:
test = [17.539342257734507, 26.04599392479185, -0.32854065948368616]
np.exp(test)/np.sum(np.exp(test))

array([2.02078612e-04, 9.99797921e-01, 3.51234685e-12])

# iterative

In [95]:
%%cython
import numpy as np
import random 
from datetime import datetime

from bernstein_race.IEBLR import IEBLR2

cdef float epsilon, delta, sig
cdef int n_max, n_jobs, i_max

epsilon = 0.01
delta = 0.05

def alg2(x0, symulation, delta = 0.01, epsilon = 0.05, n_max = 208, n_jobs = 4, i_max = None):
    cdef int n, i, iterations 
    sig = 1
    x0 = np.array(x0)
    x = x0
    n = len(x0)
    i = 0
    iterations = 0
    if not i_max:
        i_max = 0
    with open("x_history_alg_2", "w+") as file:
        while 1:
            child = [min(700, x_i + sig * np.random.normal()) for x_i in x] #python limitation
            race = IEBLR2(symulation, child, x, 
                            delta = delta, epsilon = epsilon, n_max = n_max, n_jobs = n_jobs)
            if race == 1:
                x = child
                sig = min(10, 2*sig) #ograniczamy wariancje
                today = datetime.now()
                ps = np.exp(x)/np.sum(np.exp(x))
                i+=1
                file.write(f"{i}, {x}, {today} \n")
            else:
                sig = 0.84*sig
            iterations+=1
            if iterations > i_max:
                break   

In [96]:
from games.war3par.war import symulation
alg2([1,1,1], symulation, i_max = 50)

# cov

In [80]:
%%cython
import numpy as np
import random 
from datetime import datetime

from bernstein_race.IEBLR import IEBLR2

cdef float epsilon, delta, sig
cdef int n_max, n_jobs, i_max

epsilon = 0.01
delta = 0.05

def alg3(x0, symulation, delta = 0.01, epsilon = 0.05, n_max = 208, n_jobs = 4, i_max = None):
    cdef int n, i, iterations 
    sig = 1
    x0 = np.array(x0)
    P = [x0]
    n = len(x0)
    i = 0
    iterations = 0
    if not i_max:
        i_max = 0
    with open("x_history_alg_3", "w+") as file:
        while 1:
            for ancestor in P:
                child = [min(700, x_i + sig * np.random.normal()) for x_i in ancestor] #python limitation
                race = IEBLR2(symulation, child, ancestor, 
                            delta = delta, epsilon = epsilon, n_max = n_max, n_jobs = n_jobs)
                if race == 2:
                    break
            if race == 1:
                P.append(child)
                sig = min(10, 2*sig) #ograniczamy wariancje
                today = datetime.now()
                i+=1
                file.write(f"{i}, {child}, {today} \n")
            else:
                sig = 0.84*sig
            iterations+=1
            if iterations > i_max:
                break   

    

In [79]:
from games.war3par.war import symulation
alg3([1,1,1], symulation, i_max = 20)

TypeError: Expected list, got numpy.ndarray

# aprox cov

In [82]:
%%cython
import numpy as np
import random 
from datetime import datetime

from bernstein_race.IEBLR import IEBLR2

cdef float epsilon, delta, sig
cdef int n_max, n_jobs, i_max

epsilon = 0.01
delta = 0.05

def alg4(x0, symulation, delta = 0.01, epsilon = 0.05, n_max = 208, n_jobs = 4, i_max = None):
    cdef int n, i, iterations 
    sig = 1
    x0 = np.array(x0)
    P = [x0]
    n = len(x0)
    i = 0
    iterations = 0
    if not i_max:
        i_max = 0
    with open("x_history_alg_4", "w+") as file:
        while 1:
            ancestor = random.choice(P)
            child = [min(700, x_i + sig * np.random.normal()) for x_i in ancestor] #python limitation
            race = IEBLR2(symulation, child, ancestor, 
                        delta = delta, epsilon = epsilon, n_max = n_max, n_jobs = n_jobs)
            if race == 1:
                P.append(child)
                sig = min(10, 2*sig) #ograniczamy wariancje
                today = datetime.now()
                i+=1
                file.write(f"{i}, {child}, {today} \n")
            else:
                sig = 0.84*sig
            iterations+=1
            if iterations > i_max:
                break   

    

In [84]:
from games.war3par.war import symulation
alg4([1,1,1], symulation, i_max = 1000)

KeyboardInterrupt: 

In [85]:
180**2,135**2

(32400, 18225)